In [ ]:
import os
import json
import jsonlines
import argparse
import unicodedata
from pathlib import Path
from distinct import Distinct



dataset_path = Path( '/data/pretrain_data/')
new_dataset_path = Path("/data/pretrain_data/norm_data/")

# data_files = [file.name for file in dataset_path.glob("*.jsonl")]
data_files = ['dedup_cleaned_route_final.jsonl']
total_length = len(data_files)
for file in data_files:
    print('norm:', file)
    raw_len = 0
    idx = 0
    tmp_list = []
    
    file_path = os.path.join(dataset_path, file)
    with open(file_path, 'r', encoding="utf-8") as fcc_file:
    #     data = json.load(fcc_file)

    #     for d in data:
        for data in jsonlines.Reader(fcc_file):
            d=data
            raw_len += 1
            if data['zh_ratio'] > 0.6 and d['distinct'] > 0.6 and data['length'] > 200 :
                norm_d = {'dataType':data['dataType'],
                          'content': unicodedata.normalize('NFKC', d['content']),}
                tmp_list.append(norm_d)
#             if len(tmp_list) >= 700000:
#                 print('Raw:', raw_len)
#                 print('New:', len(tmp_list), 'ratio:', len(tmp_list)/raw_len)
#                 new_name = file.split('.')[0] + '_norm'+ f'_{idx}'
#                 new_name += '.json'
#                 new_file_path = os.path.join(new_dataset_path, new_name)
#                 with open(new_file_path, "w", encoding="utf-8") as dump_f:
#                     json.dump(tmp_list, dump_f, ensure_ascii=False)
#                 raw_len = 0
#                 tmp_list = []
#                 idx += 1

    print('Raw:', raw_len)
    print('New:', len(tmp_list), 'ratio:', len(tmp_list)/raw_len)
    new_name = file.split('.')[0] + '_norm' #+ f'_{idx}'
    new_name += '.json'
    new_file_path = os.path.join(new_dataset_path, new_name)
    with open(new_file_path, "w", encoding="utf-8") as dump_f:
        json.dump(tmp_list, dump_f, ensure_ascii=False)


norm: dedup_cleaned_route_final.jsonl
Raw: 276129
New: 203445 ratio: 0.7367752028942994

In [ ]:
from datasets import concatenate_datasets, load_dataset
data_file = '/data/share_user/zzd/data/pretrain_data/norm_data/' + new_name
raw_dataset = load_dataset("json", data_files=data_file, keep_in_memory=False)